In [112]:
import mfem.ser as mfem
import numpy as np
from glvis import glvis
import matplotlib.pyplot as plt
import mpi4py.MPI as MPI
print(MPI.Get_version())
import math
import h5py
import os

(4, 1)


In [ ]:
#EKLENDIII!!!

def create_2D_mesh(nx, ny, x_start, x_end, y_start, y_end):
    """
    Creates a 2D mesh with the specified intervals and coordinate ranges.

    Args:
        nx (int): Number of intervals in the x-direction.
        ny (int): Number of intervals in the y-direction.
        x_start (float): Starting x-coordinate.
        x_end (float): Ending x-coordinate.
        y_start (float): Starting y-coordinate.
        y_end (float): Ending y-coordinate.

    Returns:
        mesh: The updated mesh with vertex coordinates set accordingly.
    """
    # Generate equally spaced coordinates for x and y
    x_coords = np.linspace(x_start, x_end, nx + 1)
    y_coords = np.linspace(y_start, y_end, ny + 1)
    
    # Create the mesh with initial x and y values set to 0 (they will be updated)
    mesh = mfem.Mesh(nx, ny, "QUADRILATERAL", True, 0.0, 0.0)
    
    # Retrieve the vertex array from the mesh
    verts = mesh.GetVertexArray()
    
    # Check if the number of vertices is as expected: (nx+1) * (ny+1)
    expected_num = (nx + 1) * (ny + 1)
    num_verts = mesh.GetNV()
    
    if num_verts != expected_num:
        print("Warning: Unexpected number of vertices! ({} != {})".format(num_verts, expected_num))
    
    # Update the vertex coordinates; vertices are stored in row-major order
    k = 0
    for j in range(ny + 1):
        for i in range(nx + 1):
            verts[k][0] = x_coords[i]
            verts[k][1] = y_coords[j]
            k += 1
    
    return mesh

#
class BoundaryFlowIntegrator(mfem.LinearFormIntegrator):
    """
    Custom boundary integrator for imposing inflow boundary conditions.

    This integrator adds a boundary term of the form (v · n) g,
    where g is the prescribed inflow.
    """
    def __init__(self, inflow_coeff, ang_coeff):
        super(BoundaryFlowIntegrator, self).__init__()
        self.inflow_coeff = inflow_coeff
        self.ang_coeff = ang_coeff

    def AssembleFaceVector(self, el, el_dof, tr):
        """
        Assemble the face vector contribution on a boundary face.

        Args:
            el (mfem.ElementTransformation): Element transformation.
            el_dof (mfem.Vector): Local DOF vector (unused).
            tr (mfem.FaceElementTransformations): Face transformation.

        Returns:
            mfem.Vector: The assembled face vector.
        """
        ir = mfem.IntRules.Get(tr.Elem1.GetGeometry(), tr.Face1.GetOrder())
        dof = el_dof.Size()
        fv = mfem.Vector(dof)
        fv.Assign(0.0)
        for i in range(ir.GetNPoints()):
            ip = ir.IntPoint(i)
            weight = ip.weight * tr.Face1.Weight()
            # Evaluate inflow value g and velocity at the integration point.
            g_val = self.inflow_coeff.Eval(tr.Face1, ip)
            v_val = self.ang_coeff.Eval(tr.Face1, ip)
            # Get the face normal (as a Python list)
            n = tr.Face1.GetNormal()
            # Compute dot(v, n)
            vn = sum(v_val[j] * n[j] for j in range(len(n)))
            fv.Add(weight * vn * g_val)
        return fv
    
#EKLENDIII!!!
class SimpleCoefficient(mfem.PyCoefficient):
    def __init__(self, data, E_start, E_end):
        super(SimpleCoefficient, self).__init__()
        self.data = data  
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        # x[1] is the energy value
        E = self.E_start - x[1] * (self.E_start - self.E_end)
        # Determine the group based on the energy
        group = min(len(self.data) - 1, int((E - self.E_end) / (self.E_start - self.E_end) * len(self.data)))
        return float(self.data[group])

def assemble_system(fes, mesh, E_range, xs_t_arr, S_arr, Q_func, inflow_func=None):
    """
    Assemble the DG system for the CSDA transport equation.

    The system includes:
      - Spatial and energy convection terms (using ConvectionIntegrator)
      - Total cross section term (MassIntegrator with xs_t)
      - Interior face terms for upwinding (DGTraceIntegrator)
      - Domain source term (DomainLFIntegrator for Q)
      - Optional boundary inflow (using custom BoundaryFlowIntegrator)

    Args:
        fes (mfem.FiniteElementSpace): Finite element space.
        mesh (mfem.Mesh): Computational mesh.
        E_range (tuple): (E_start, E_end) for energy mapping.
        xs_t_arr (numpy.array): Array of total cross-section values per energy group.
        S_arr (numpy.array): Array of stopping power values per energy group.
        Q_func (callable): Function Q(x,E) for the source term.
        inflow_func (callable, optional): Function g(x,E) for inflow boundary.

    Returns:
        A (mfem.SparseMatrix): Assembled system matrix.
        b (mfem.Vector): Assembled right-hand side vector.
    """
    
    # Create a bilinear form for the system matrix.
    a = mfem.BilinearForm(fes)

    E_end, E_start = E_range

    # Define xs_t coefficient (mapped from y to energy)
    def xs_t_coeff_func(x):
        y = x[1]
        E = E_start - y * (E_start - E_end)
        nE = len(xs_t_arr)
        # Uniform group mapping.
        group = min(len(xs_t_arr) - 1, int(x[1] * len(xs_t_arr)))
        return float(xs_t_arr[group])
    
    xs_t_coeff = mfem.PyCoefficient(xs_t_coeff_func)


    # Define stopping power coefficient (for energy convection)
    def stopping_power_coeff_func(x):
        y = x[1]
        E = E_start - y * (E_start - E_end)
        nE = len(S_arr)
        group = min(nE - 1, int((E_start - E) / (E_start - E_end) * nE))
        return S_arr[group]

    class StoppingPowerVectorCoefficient(mfem.VectorCoefficient):
        def __init__(self):
            super(StoppingPowerVectorCoefficient, self).__init__(mesh.Dimension())
        def Eval(self, V, x):
            V.SetSize(mesh.Dimension())
            # For energy convection, set velocity = (0, S(E))
            V[0] = 0.0
            V[1] = stopping_power_coeff_func(x)
    S_vector_coeff = StoppingPowerVectorCoefficient()

    # Define spatial velocity coefficient.
    class VelocityVectorCoefficient(mfem.VectorCoefficient):
        def __init__(self):
            super(VelocityVectorCoefficient, self).__init__(mesh.Dimension())
        def Eval(self, V, x):
            V.SetSize(mesh.Dimension())
            # For spatial convection, assume constant velocity in x-direction.
            V[0] = 1.0
            V[1] = 0.0
    velocity_coeff = VelocityVectorCoefficient()

    # Add domain integrators for convection in space and energy.
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(velocity_coeff, 1.0))
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(S_vector_coeff, 1.0))
    # Add mass integrator for absorption (xs_t).
    a.AddDomainIntegrator(mfem.MassIntegrator(xs_t_coeff))

    # Add interior face integrators for DG upwind flux.
    a.AddInteriorFaceIntegrator(mfem.DGTraceIntegrator(velocity_coeff, 1.0, -0.5))
    a.AddInteriorFaceIntegrator(mfem.DGTraceIntegrator(S_vector_coeff, 1.0, -0.5))

    a.Assemble()
    A = a.SpMat()

    # Assemble the right-hand side: domain source from Q(x,E)
    b = mfem.LinearForm(fes)
    Q_coeff = mfem.FunctionCoefficient(lambda x: Q_func(x[0], E_start - x[1]*(E_start - E_end)))
    b.AddDomainIntegrator(mfem.DomainLFIntegrator(Q_coeff))

    # If an inflow function is provided, add a boundary integrator.
    if inflow_func is not None:
        inflow_coeff = mfem.FunctionCoefficient(lambda x: inflow_func(x[0], E_start - x[1]*(E_start - E_end)))
        # Assume the left boundary (attribute 1) is the inflow.
        b.AddBoundaryIntegrator(BoundaryFlowIntegrator(inflow_coeff, velocity_coeff), 1)
    b.Assemble()

    return A, b

class TransportSolver:
    """
    Class for solving the CSDA transport equation using various methods.
    
    Provides both a direct LU solver and a source iteration solver.
    """
    def __init__(self, A, b, fes):
        """
        Initialize the solver with system matrix, RHS vector, and FE space.

        Args:
            A (mfem.SparseMatrix): System matrix.
            b (mfem.Vector): Right-hand side vector.
            fes (mfem.FiniteElementSpace): Finite element space.
        """
        self.A = A
        self.b = b
        self.fes = fes

    def solve_lu(self):
        """
        Solve the linear system using a direct LU solver.

        Returns:
            mfem.Vector: The solution vector.
        """
        x = mfem.Vector(self.b.Size())
        lu_solver = mfem.SuperLUSolver()
        lu_solver.SetOperator(self.A)
        lu_solver.Mult(self.b, x)
        return x

    def solve_source_iteration(self, xs_s_coeff, max_iter=100, tol=1e-6):
        """
        Solve the transport equation using source iteration.

        This method treats the scattering term explicitly and iterates until
        the relative change in the solution is below the specified tolerance.

        Args:
            xs_s_coeff (mfem.Coefficient): Coefficient for the scattering cross-section.
            max_iter (int): Maximum number of iterations.
            tol (float): Convergence tolerance.

        Returns:
            mfem.Vector: The converged solution vector.
        """
        # Initialize solution as a GridFunction in the FE space.
        x = mfem.GridFunction(self.fes)
        x.Assign(0.0)
        # Assemble scattering operator: mass integrator with xs_s.
        scatter_bform = mfem.BilinearForm(self.fes)
        scatter_bform.AddDomainIntegrator(mfem.MassIntegrator(xs_s_coeff))
        scatter_bform.Assemble()
        ScatterMat = scatter_bform.SpMat()

        # LU solver for the transport sweep.
        lu_solver = mfem.SuperLUSolver()
        lu_solver.SetOperator(self.A)

        b_iter = mfem.Vector(self.b.Size())
        for it in range(max_iter):
            # Compute scattering source: ScatterMat * x.
            scatter_source = mfem.Vector(self.b.Size())
            ScatterMat.Mult(x, scatter_source)
            # New RHS: b_new = b + scattering source.
            b_iter.Assign(self.b)
            b_iter += scatter_source
            # Solve the transport equation: A * x_new = b_new.
            x_new = mfem.Vector(self.b.Size())
            lu_solver.Mult(b_iter, x_new)
            # Check convergence.
            diff = mfem.Vector(x_new)
            diff -= x
            norm_diff = diff.Norml2()
            norm_x = x.Norml2()
            if norm_x > 1e-10 and norm_diff / norm_x < tol:
                x.Assign(x_new)
                print("Source iteration converged in", it + 1, "iterations.")
                return x
            x.Assign(x_new)
        print("Source iteration did not converge in", max_iter, "iterations.")
        return x
    
#EKLENDIII!!!

def glvis_visualize_2D(mesh, solution, title="CSDA Transport Solution"):
    """
    Visualize the solution using GLVis.

    Args:
        mesh (mfem.Mesh): The computational mesh.
        solution (mfem.GridFunction): The solution grid function.
        title (str): Title for the GLVis window.
    """
    ss = mfem.socketstream("localhost", 19916)
    if not ss.good():
        print("Unable to open GLVis socket connection.")
        return
    ss.send_text("solution\n" + title + "\n")
    mesh.Print(ss)
    solution.Save(ss)
    ss.send_text("\n")

#EKLENDIII!!!

def matplotlib_visualize_2D(mesh, solution, E_range, nx=20, nE=50):
    """
    Visualize the solution using matplotlib.

    Assumes the mesh is a structured Cartesian mesh.
    The y-coordinate is mapped to energy using:
         E = E_start - y*(E_start - E_end).

    Args:
        mesh (mfem.Mesh): The computational mesh.
        solution (mfem.GridFunction): The solution grid function.
        E_range (tuple): (E_end, E_start) for energy mapping.
        nx (int): Number of spatial elements.
        nE (int): Number of energy groups.
    """
    # Get the mesh nodes.
    nodes = mesh.GetNodes()
    if nodes.Size() == 0:
        print("Mesh has no nodes; cannot plot.")
        return
    # nodes is a mfem.Matrix with dimensions (dim x num_nodes).
    dim = nodes.Width()
    num_nodes = nodes.Height()
    # Extract x and y coordinates.
    x_coords = np.array([nodes.Get(i, 0) for i in range(num_nodes)])
    y_coords = np.array([nodes.Get(i, 1) for i in range(num_nodes)])
    # Map y-coordinate to energy.
    E_end, E_start = E_range
    E_values = E_start - y_coords * (E_start - E_end)
    # Get solution values at nodes.
    sol = np.array([solution.GetValue(i) for i in range(num_nodes)])
    # Create a scatter plot.
    plt.figure()
    sc = plt.scatter(x_coords, E_values, c=sol, cmap='viridis')
    plt.xlabel('x')
    plt.ylabel('Energy (MeV)')
    plt.title('CSDA Transport Solution')
    plt.colorbar(sc, label='Flux')
    plt.show()

#EKLENDIII!!!

def read_data(nE):
    """
    Read cross-section and stopping power data from data/data.h5.

    The HDF5 file must contain datasets named:
        "xs_t_{nE}", "xs_s_{nE}", and "S_{nE}"

    Args:
        nE (int): Number of energy groups.

    Returns:
        xs_t_arr (numpy.array): Total cross-section array.
        xs_s_arr (numpy.array): Scattering cross-section array.
        S_arr (numpy.array): Stopping power array.
    """
    data_file = "data.h5"
    with h5py.File(data_file, "r") as f:
        xs_t_arr = f[f"xs_t_{nE}"][:]
        xs_s_arr = f[f"xs_s_{nE}"][:]
        S_arr = f[f"S_{nE}"][:]
    return xs_t_arr, xs_s_arr, S_arr

#EKLENDIII!!!

def Q_function(x, E):
    """
    Define the source term Q(x,E).

    For this example, a zero source is used.

    Args:
        x (float): Spatial coordinate.
        E (float): Energy.

    Returns:
        float: Source term value.
    """
    return 0.0

#EKLENDIII!!!

def inflow_function(x, E):
    """
    Define the inflow boundary condition g(x,E) at x = 0.

    For this example, a unit flux is imposed at 1 MeV.

    Args:
        x (float): Spatial coordinate.
        E (float): Energy.

    Returns:
        float: Inflow flux value.
    """
    if abs(E - 1.0) < 1e-3:
        return E
    return 0.0




In [175]:

def main():
    # Problem parameters
    nx = 20
    nE = 50
    x_start = 0.0
    x_end = 0.3
    E_start = 1.0
    E_end = 0.01
    E_range = E_start, E_end

    # Read cross-section and stopping power data from HDF5.
    xs_t_arr, xs_s_arr, S_arr = read_data(nE)

    # Create mesh and get energy range mapping.
    mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)

    # Define a DG finite element space (L2) of order 1.
    order = 1
    fec = mfem.L2_FECollection(order, mesh.Dimension())
    fes = mfem.FiniteElementSpace(mesh, fec)
    print("Number of unknowns:", fes.GetVSize())

    # Assemble the system matrix and RHS vector.
    A, b = assemble_system(fes, mesh, E_range, xs_t_arr, S_arr, Q_function, inflow_function)

    # Build scattering coefficient (xs_s) for source iteration.
    def xs_s_coeff_func(x):
        y = x[1]
        E = E_start - y * (E_start - E_end)
        nE = len(xs_t_arr)
        # Uniform group mapping.
        group = min(nE - 1, int((E_start - E) / (E_start - E_end) * nE))
        return float(xs_s_arr[group])
    
    xs_s_coeff = mfem.FunctionCoefficient(lambda x: xs_s_coeff_func(x))

    # Initialize the transport solver with the assembled system.
    solver = TransportSolver(A, b, fes)

    # Solve using direct LU solver.
    x_lu = solver.solve_lu()
    print("LU solver completed.")

    # Solve using source iteration.
    x_si = solver.solve_source_iteration(xs_s_coeff)
    print("Source iteration solver completed.")

    # Create GridFunctions to hold the solutions.
    sol_gf_lu = mfem.GridFunction(fes)
    sol_gf_lu.Assign(x_lu)
    sol_gf_si = mfem.GridFunction(fes)
    sol_gf_si.Assign(x_si)

    # Save the solution using MFEM's VisItDataCollection (outputs HDF5 files).
    dc = mfem.VisItDataCollection("csda_solution", mesh)
    dc.RegisterField("flux", sol_gf_si)
    dc.Save()
    print("Solution saved to csda_solution files.")

    # Visualize the solution with GLVis.
    glvis_visualize(mesh, sol_gf_si, "CSDA Transport Solution (Source Iteration)")

    # Visualize the solution with matplotlib.
    matplotlib_visualize(mesh, sol_gf_si, E_range, nx, nE)

if __name__ == "__main__":
    main()


Number of unknowns: 4000


TypeError: PyCoefficient.__init__() takes 1 positional argument but 2 were given

In [ ]:
class AngularFluxCoefficient(mfem.PyCoefficient):
    """
    Angular flux coefficient for discrete ordinates using Gauss-Legendre quadrature.
    
    This coefficient defines the angular dependence of the flux. It maps the angular
    variable (assumed to be x[0] in the integration point, representing the cosine of the angle)
    to a prescribed angular flux value.
    
    For example, a simple angular flux distribution is defined as:
        psi(mu) = 1 + mu,
    where mu is in the interval [-1, 1]. This provides a non-uniform angular distribution.
    
    This coefficient is intended to be used in angular discretization where a Gauss-Legendre
    quadrature rule of order p=6 is applied.
    
    Attributes:
        None.
    """
    
    def EvalValue(self, x):
        """
        Evaluates the angular flux at a given angular coordinate.
        
        Args:
            x (mfem.Vector): The coordinate vector, where x[0] is assumed to be the angular variable (mu).
            
        Returns:
            float: The angular flux value at the given coordinate.
        """
        # Assume x[0] is the angular variable (mu) in the range [-1, 1]
        mu = x[0]
        # Define a simple angular flux distribution, e.g., psi(mu) = 1 + mu
        return 1.0 + mu

'''
# Usage Example:
# Create an instance of AngularFluxCoefficient.
angular_flux_coeff = AngularFluxCoefficient()

# When integrating over the angular variable, use Gauss-Legendre quadrature with p=6.
# For example, to obtain the integration rule for a segment (1D interval):
IR = mfem.IntRules.Get(mfem.Geometry.SEGMENT, 6)
# IR now contains the 6 Gauss-Legendre quadrature points and weights which can be used
# in the assembly of integrals involving the angular variable.

'''

In [1]:
def create_2D_mesh(nx, ny, x_start, x_end, y_start, y_end):
    """
    Creates a 2D mesh with the specified intervals and coordinate ranges.

    Args:
        nx (int): Number of intervals in the x-direction.
        ny (int): Number of intervals in the y-direction.
        x_start (float): Starting x-coordinate.
        x_end (float): Ending x-coordinate.
        y_start (float): Starting y-coordinate.
        y_end (float): Ending y-coordinate.

    Returns:
        mesh: The updated mesh with vertex coordinates set accordingly.
    """
    # Generate equally spaced coordinates for x and y
    x_coords = np.linspace(x_start, x_end, nx + 1)
    y_coords = np.linspace(y_start, y_end, ny + 1)
    
    # Create the mesh with initial x and y values set to 0 (they will be updated)
    mesh = mfem.Mesh(nx, ny, "QUADRILATERAL", True, 0.0, 0.0)
    
    # Retrieve the vertex array from the mesh
    verts = mesh.GetVertexArray()
    
    # Check if the number of vertices is as expected: (nx+1) * (ny+1)
    expected_num = (nx + 1) * (ny + 1)
    num_verts = mesh.GetNV()
    
    if num_verts != expected_num:
        print("Warning: Unexpected number of vertices! ({} != {})".format(num_verts, expected_num))
    
    # Update the vertex coordinates; vertices are stored in row-major order
    k = 0
    for j in range(ny + 1):
        for i in range(nx + 1):
            verts[k][0] = x_coords[i]
            verts[k][1] = y_coords[j]
            k += 1
    
    return mesh

def gauss_legendre_dirs(N_dir):
    """
    Compute Gauss-Legendre quadrature points (mu_i) and weights (w_i)
    for discrete ordinates. Typically used for [-1,1].
    Returns (mu[], w[]) each of length N_dir.
    """
    mu = []
    w  = []

    gg_mu, gg_w = np.polynomial.legendre.leggauss(N_dir)
    
    mu = gg_mu.tolist()
    w  = gg_w.tolist()
    return mu, w

class TotalXSCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the total cross-section Σ_t(E).

    Maps the normalized energy coordinate (x[1]) in [0,1] to the corresponding 
    cross-section value. Here, y=0 corresponds to E = E_start and y=1 corresponds to 
    E = E_end.
    """
    def __init__(self, xs_t_data, E_start, E_end):
        super(TotalXSCoefficient, self).__init__()
        self.xs_t_data = xs_t_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        # x[1] is the normalized energy coordinate in [0,1]
        y = x[1]
        # Map y ∈ [0,1] to energy E ∈ [E_end, E_start]
        E = self.E_start + y * (self.E_start - self.E_end)
        n_groups = len(self.xs_t_data)
        # Hesaplama: (E - E_end)/(E_start - E_end) gives a fraction in [0,1];
        # bunu n_groups ile çarparak hangi gruba denk geldiğini buluyoruz.
        group = min(n_groups - 1, int((E - self.E_end) / (self.E_start - self.E_end) * n_groups))
        return float(self.xs_t_data[group])
    
class ScatteringXSCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the scattering cross-section Σ_s(E).

    Maps the normalized energy coordinate (x[1] in [0,1]) to the corresponding 
    scattering cross-section value. Here, y=0 corresponds to E = E_start and y=1 
    corresponds to E = E_end.
    """
    def __init__(self, xs_s_data, E_start, E_end):
        super(ScatteringXSCoefficient, self).__init__()
        self.xs_s_data = xs_s_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        y = x[1]
        E = self.E_start + y * (self.E_start - self.E_end)
        n_groups = len(self.xs_s_data)
        group = min(n_groups - 1, int((E - self.E_end) / (self.E_start - self.E_end) * n_groups))
        return float(self.xs_s_data[group])

class StoppingPowerCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the stopping power S(E).

    Maps the normalized energy coordinate (x[1]) in [0,1] to the corresponding 
    S(E) value. Here, y=0 corresponds to E = E_start and y=1 corresponds to 
    E = E_end.
    """
    def __init__(self, S_data, E_start, E_end):
        super(StoppingPowerCoefficient, self).__init__()
        self.S_data = S_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        y = x[1]
        # Aynı şekilde, y in [0,1] → E in [E_end, E_start]
        E = self.E_start + y * (self.E_start - self.E_end)
        n_groups = len(self.S_data)
        group = min(n_groups - 1, int((E - self.E_end) / (self.E_start - self.E_end) * n_groups))
        return float(self.S_data[group])
    

class StoppingPowerDerivativeCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the derivative of the stopping power S(E), i.e., S'(E).

    Maps the normalized energy coordinate (x[1] in [0,1]) to the corresponding 
    derivative value. Here, y=0 corresponds to E = E_start and y=1 corresponds to 
    E = E_end.
    """
    def __init__(self, dS_data, E_start, E_end):
        super(StoppingPowerDerivativeCoefficient, self).__init__()
        self.dS_data = dS_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        y = x[1]
        E = self.E_start + y * (self.E_start - self.E_end)
        n_groups = len(self.dS_data)
        group = min(n_groups - 1, int((E - self.E_end) / (self.E_start - self.E_end) * n_groups))
        return float(self.dS_data[group])

nx = 20
nE = 50
x_start = 0.0
x_end = 0.3
E_start = 1.0
E_end = 0.01
N_ang = 8
mu_vals, w_vals = np.polynomial.legendre.leggauss(N_ang)
order = 1
data = h5py.File("data.h5", "r")
E_vals    = data[f"E_{nE}"][:]
xs_t_vals = data[f"xs_t_{nE}"][:] 
xs_s_vals = data[f"xs_s_{nE}"][:] 
S_vals = data[f"S_{nE}"][:] 
mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)
dim = mesh.Dimension()
print(dim)


fec = mfem.DG_FECollection(order, dim)
fes = mfem.FiniteElementSpace(mesh, fec)
ndofs = fes.GetVSize()
print(f"Number of finite element unknowns:: {ndofs}")
a = mfem.BilinearForm(fes)

data = h5py.File("data.h5", "r")
E_vals    = data[f"E_{nE}"][:]
xs_t_vals = data[f"xs_t_{nE}"][:] 
xs_s_vals = data[f"xs_s_{nE}"][:] 
S_vals = data[f"S_{nE}"][:] 

ess_tdof_list = mfem.intArray()
dir_bdr = mfem.intArray(mesh.bdr_attributes.Max())
dir_bdr.Assign(1)
dir_bdr.Size()

print(dir_bdr.Size())
print(ess_tdof_list.Size())

class TotalXSCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the total cross-section Σ_t(E).

    Maps the normalized energy coordinate (x[1]) in [0,1] to the corresponding 
    cross-section value. Here, y=0 corresponds to E = E_start and y=1 corresponds to 
    E = E_end.
    """
    def __init__(self, xs_t_data, E_start, E_end):
        super(TotalXSCoefficient, self).__init__()
        self.xs_t_data = xs_t_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        y = x[1]  # normalized energy coordinate in [0,1]
        # Map y ∈ [0,1] to energy E ∈ [E_start, E_end]
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.xs_t_data)
        # Fraction: (E - E_start)/(E_end - E_start)
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.xs_t_data[group])
    
class ScatteringXSCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the scattering cross-section Σ_s(E).

    Maps the normalized energy coordinate (x[1]) in [0,1] to the corresponding 
    scattering cross-section value. Here, y=0 corresponds to E = E_start and y=1 
    corresponds to E = E_end.
    """
    def __init__(self, xs_s_data, E_start, E_end):
        super(ScatteringXSCoefficient, self).__init__()
        self.xs_s_data = xs_s_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        y = x[1]
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.xs_s_data)
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.xs_s_data[group])

class StoppingPowerCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the stopping power S(E).

    Maps the normalized energy coordinate (x[1]) in [0,1] to the corresponding 
    S(E) value. Here, y=0 corresponds to E = E_start and y=1 corresponds to 
    E = E_end.
    """
    def __init__(self, S_data, E_start, E_end):
        super(StoppingPowerCoefficient, self).__init__()
        self.S_data = S_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        y = x[1]
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.S_data)
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.S_data[group])
    
class StoppingPowerDerivativeCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the derivative of the stopping power S(E), i.e., S'(E).

    Maps the normalized energy coordinate (x[1]) in [0,1] to the corresponding 
    derivative value. Here, y=0 corresponds to E = E_start and y=1 corresponds to 
    E = E_end.
    """
    def __init__(self, dS_data, E_start, E_end):
        super(StoppingPowerDerivativeCoefficient, self).__init__()
        self.dS_data = dS_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        y = x[1]
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.dS_data)
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.dS_data[group])


NameError: name 'mfem' is not defined